In [1]:
import cv2 
import tensorflow as tf
from keras.models import load_model
import pickle
new_model = load_model('emotion3.h5')
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("face-trainner.yml")

labels = {"person_name": 1}
with open("face-labels.pickle", 'rb') as f:
	og_labels = pickle.load(f)
	labels = {v:k for k,v in og_labels.items()}

label_dict = {0:'Angry',1:'Disgust',2:'Fear',3:'Happy',4:'Neutral',5:'Sad',6:'Surprise'}

def load_image(filename):
	img = load_img(filename, grayscale=True, target_size=(48, 48))
	img = img_to_array(img)
	img = img.reshape(1, 48, 48, 1)
	img = img.astype('float32')
	img = img / 255.0
	return img

face_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_frontalface_alt2.xml')

vid = cv2.VideoCapture(0) 
  
while(True): 
      
    ret, frame = vid.read() 
    cv2.imwrite("a.jpg", frame)
    img=load_image('a.jpg')
    result = new_model.predict(img)
    result = list(result[0])
    img_index = result.index(max(result))
    pred=str(label_dict[img_index])
    font = cv2.FONT_HERSHEY_SIMPLEX 
  
    org = (20, 50) 

    fontScale = 1

    color = (255, 255, 0) 

    thickness = 2




    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    for (x, y, w, h) in faces:
    	roi_gray = gray[y:y+h, x:x+w] 
    	roi_color = frame[y:y+h, x:x+w]
    	id_, conf = recognizer.predict(roi_gray)
    	if conf>=4 and conf <= 85:
    		
    		font = cv2.FONT_HERSHEY_SIMPLEX
    		name = labels[id_]
    		color = (255, 255, 255)
    		stroke = 2
    		cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)

    	img_item = "7.png"
    	cv2.imwrite(img_item, roi_color)

    	color = (255, 0, 0)  
    	stroke = 2
    	end_cord_x = x + w
    	end_cord_y = y + h
    	cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, stroke)
    
    cv2.putText(frame, pred, org, font,  
                       fontScale, (0,255,0), thickness, cv2.LINE_AA) 
    cv2.imshow('frame',frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
  
vid.release() 
cv2.destroyAllWindows()

C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
